In [10]:
import pandas as pd

columns = ['target', 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
train_data = pd.read_csv('adult.train.10k.discrete', header=None)
train_data.columns = columns

for col in columns:
    train_data[col] = train_data[col].str.strip()
X_train = train_data.iloc[:, 1:]
y_train = train_data.iloc[:, 0]

test_data = pd.read_csv('adult.test.10k.discrete', header=None)
test_data.columns = columns

for col in columns:
    test_data[col] = test_data[col].str.strip()
X_test = test_data.iloc[:, 1:]
y_test = test_data.iloc[:, 0]

In [71]:
import numpy as np

class Node:
    def __init__(self, value=None, feature=None, children=None):
        self.value = value
        self.feature = feature
        self.children = children


def calculate_entropy(X):
    # H(S) = - sigma(p*log(p))
    e = 0
    labels = X.unique()
    for l in labels:
        p = len(X[X==l]) / len(X)
        e -= p * np.log2(p) if p > 0 else 0
    return e


def calculate_gain(X, y, feature):
    total_entropy = calculate_entropy(y)
    conditional_entropy = 0

    unique_values = X[feature].unique()
    for value in unique_values:
        # find all rows with the specific value
        subset = X[X[feature]==value]
        subset_labels = y[X[feature]==value]
        if not subset.empty:
            e = calculate_entropy(subset_labels)
            conditional_entropy += e * len(subset)/len(X)

    return total_entropy - conditional_entropy


def find_most_informative(X, y, features):
    most_gain = -float("inf")
    best_feature = None

    for feature in features:
        g = calculate_gain(X, y, feature)
        if g > most_gain:
            best_feature = feature
            most_gain = g

    return best_feature

def most_common_class(y):
    classes, counts = np.unique(y, return_counts=True)
    idx =  np.argmax(counts)
    return classes[idx]

def build_tree(X, y, features_left):

    # base case
    if len(y.unique()) == 1:
        return Node(value=y.iloc[0])

    if len(features_left) == 0 or len(X) == 0:
        return Node(value=most_common_class(y))

    best_feature = find_most_informative(X, y, features_left)
    unique_values = X[best_feature].unique()

    children = {}
    for val in unique_values:
        # find all rows which their feature value is 'val'
        subset = X[X[best_feature]==val]
        labels = y[X[best_feature]==val]
        if not subset.empty:
            new_features = [f for f in features_left if f != best_feature]
            children[val] = build_tree(subset, labels, new_features)

    return Node(feature=best_feature, children=children)

def predict(tree, data):
    try:
        if tree.value:
            return tree.value

        feature = tree.feature
        value = data[feature]
        return predict(tree.children[value], data)

    except:
        return '<=50K'

In [70]:
root = build_tree(X_train, y_train, list(X_train.columns))

In [73]:
from sklearn.metrics import classification_report, accuracy_score

preds_train = []
for i in range(len(X_train)):
    preds_train.append(predict(root, X_train.iloc[i]))

print(accuracy_score(y_train, preds_train))
print(classification_report(y_train, preds_train))

0.8754
              precision    recall  f1-score   support

       <=50K       0.90      0.94      0.92      7550
        >50K       0.79      0.67      0.73      2450

    accuracy                           0.88     10000
   macro avg       0.84      0.81      0.82     10000
weighted avg       0.87      0.88      0.87     10000



In [74]:
preds_test = []
for i in range(len(X_test)):
    preds_test.append(predict(root, X_test.iloc[i]))

print(accuracy_score(y_test, preds_test))
print(classification_report(y_test, preds_test))

0.8097
              precision    recall  f1-score   support

       <=50K       0.85      0.91      0.88      7539
        >50K       0.65      0.50      0.56      2461

    accuracy                           0.81     10000
   macro avg       0.75      0.71      0.72     10000
weighted avg       0.80      0.81      0.80     10000

